In [ ]:
pip install dataretrieval

In [ ]:
from dataretrieval import nwis
import pandas as pd

In [ ]:
# Get data for Fox River, WI
# Define station IDs and parameters
stations = ['04084445', '040851385']

# Parameter codes: 00060 = discharge (ft3/s), 63680 = turbidity (NTU)
params = {
    '00060': 'discharge',   # Discharge (ft3/s)
    '63680': 'turbidity'    # Turbidity (NTU)
}

# Data type: 'iv' stands for instantaneous values (15-minute interval when available)
service = 'iv'


In [ ]:
# Data storage dictionary
all_data = {}

# Loop over stations and parameters
for site in stations:
    print(f"\n=== Station {site} ===")
    all_data[site] = {}
    for param_code, param_name in params.items():
        print(f"  Downloading {param_name} ({param_code})...")
        try:
            data, meta = nwis.get_iv(
                sites=site,
                parameterCd=param_code,
                start='1900-01-01',
                end='2025-12-31'
            )
            if not data.empty:
                # Save in dict and CSV
                all_data[site][param_name] = data
                filename = f"{site}_{param_name}_iv.csv"
                data.to_csv(filename)
                print(f"    ✅ Success: {len(data)} records saved to {filename}")
            else:
                print(f"    ⚠️ No data found for {param_name}.")
        except Exception as e:
            print(f"    ❌ Error downloading {param_name}: {e}")


=== Station 04084445 ===
    ✅ Success: 1195336 records saved to 04084445_discharge_iv.csv
    ⚠️ No data found for turbidity.

=== Station 040851385 ===
    ✅ Success: 2504651 records saved to 040851385_discharge_iv.csv
    ✅ Success: 1395761 records saved to 040851385_turbidity_iv.csv


In [ ]:
# Combine everything into one long dataframe
combined_list = []

for site, params in all_data.items():
    for param, df in params.items():
        # Copy and annotate the dataframe
        df_copy = df.copy()
        df_copy['station'] = site
        combined_list.append(df_copy)

# Combine into one big dataframe
combined_df = pd.concat(combined_list)

# See what it looks like
combined_df.head()

,site_no,00060,00060_cd,station,00060_2,00060_2_cd,63680_ysi 6136,63680_ysi 6136_cd,63680,63680_cd
datetime,,,,,,,,,,
1991-10-01 05:15:00+00:00,04084445,1580.0,"A, [91]",04084445,NaN,NaN,NaN,NaN,NaN,NaN
1991-10-01 05:30:00+00:00,04084445,1580.0,"A, [91]",04084445,NaN,NaN,NaN,NaN,NaN,NaN
1991-10-01 05:45:00+00:00,04084445,1580.0,"A, [91]",04084445,NaN,NaN,NaN,NaN,NaN,NaN
1991-10-01 06:00:00+00:00,04084445,1580.0,"A, [91]",04084445,NaN,NaN,NaN,NaN,NaN,NaN
1991-10-01 06:15:00+00:00,04084445,1600.0,"A, [91]",04084445,NaN,NaN,NaN,NaN,NaN,NaN
